<a href="https://colab.research.google.com/github/artbbs/42SP-arfernan/blob/main/MyBoot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Oi", "Como vai você?", "E aí?", "Olá", "Você está bem?"],
              "responses": ["Olá!!", "Bem, e você?", "Tudo maravilha!", "Oi", "Muito melhor agora"],
             },
             {"tag": "age",
              "patterns": ["Você tem quantos anos?", "Quantos anos você tem?", "Quando é o seu aniversário?", "Quando você nasceu?"],
              "responses": ["Eu tenho 37 anos", "Eu nasci em 1983", "Meu aniversário é em dezembro", "05/12/1983"]
             },
             {"tag": "date",
              "patterns": ["Tem planos para o fim de semana?",
"O que você faz no tempo livre?", "Quais seus planos para este fim de semana?", "Vamos ao parque no fim de semana?"],
              "responses": ["Eu não tenho nada para fazer...", "Eu estou ocupado", "Pretendo dormir o tempo todo"]
             },
             {"tag": "name",
              "patterns": ["Qual o seu nome?", "Você tem apelido?", "Quem é você?", "Como você se chama?", "Quero saber seu nome"],
              "responses": ["Meu nome é Boop", "Sim, sou o Boop", "Boop."]
             },
             {"tag": "goodbye",
              "patterns": [ "Boa tarde", "Bom dia", "Boa noite", "tchau", "até mais", "obrigado", "adeus", "até"],
              "responses": ["Foi ótimo falar com você hoje", "Nos vemos depois", "Nos vemos em breve!"]
             }
]}

In [3]:
lemmatizer = WordNetLemmatizer()

In [ ]:
words = []
classes = []
doc_X = []
doc_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # adicione a tag às classes se ainda não estiver lá
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# lematizar todas as palavras do vocabulário e convertê-las em minúsculas
# se as palavras não aparecem na pontuação
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

# classificando o vocabulário e as classes em ordem alfabética e tomando o 
# conjunto para garantir que não ocorram duplicatas
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
# lista de dados de treinamento
training = []
out_empty = [0] * len(classes)

# criando o modelo do saco de palavras
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)

# marca o índice da classe que o padrão atual está associado
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1

    # adicionar um BoW codificado a quente e classes associadas ao treinamento
    training.append([bow, output_row])

# embaralhar os dados e convertê-los em um array
random.shuffle(training)
training = np.array(training, dtype=object)

# divide os recursos e rótulos de destino
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [ ]:
# definindo alguns parâmetros
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200

# o modelo de aprendizado profundo
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               6784      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 15,365
Trainable params: 15,365
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
1/1 [==============================] - 0s 2ms/step - loss: 1.6302 - accuracy: 0.1923
Epoch 2/200
1/1 

1/1 [==============================] - 0s 2ms/step - loss: 0.1004 - accuracy: 0.9615
Epoch 74/200
1/1 [==============================] - 0s 0s/step - loss: 6.9829e-04 - accuracy: 1.0000
Epoch 75/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 76/200
1/1 [==============================] - 0s 998us/step - loss: 1.2654e-04 - accuracy: 1.0000
Epoch 77/200
1/1 [==============================] - 0s 0s/step - loss: 3.5326e-04 - accuracy: 1.0000
Epoch 78/200
1/1 [==============================] - 0s 992us/step - loss: 0.0060 - accuracy: 1.0000
Epoch 79/200
1/1 [==============================] - 0s 2ms/step - loss: 6.1049e-05 - accuracy: 1.0000
Epoch 80/200
1/1 [==============================] - 0s 0s/step - loss: 0.1263 - accuracy: 0.9615
Epoch 81/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 82/200
1/1 [==============================] - 0s 997us/step - loss: 2.7314e-04 - accuracy: 1.0000
Epoch 83/2

1/1 [==============================] - 0s 0s/step - loss: 0.0173 - accuracy: 1.0000
Epoch 154/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 155/200
1/1 [==============================] - 0s 1ms/step - loss: 1.0759e-04 - accuracy: 1.0000
Epoch 156/200
1/1 [==============================] - 0s 999us/step - loss: 1.8645e-04 - accuracy: 1.0000
Epoch 157/200
1/1 [==============================] - 0s 998us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 158/200
1/1 [==============================] - 0s 1ms/step - loss: 2.5096e-04 - accuracy: 1.0000
Epoch 159/200
1/1 [==============================] - 0s 1ms/step - loss: 4.1086e-04 - accuracy: 1.0000
Epoch 160/200
1/1 [==============================] - 0s 0s/step - loss: 0.0047 - accuracy: 1.0000
Epoch 161/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 162/200
1/1 [==============================] - 0s 0s/step - loss: 2.1308e-04 - accuracy: 1.0000
Epo

In [ ]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
# executando o chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

Como vai você?
Olá!!
Você está bem?
Tudo maravilha!
Oi
Nos vemos em breve!
Quando é o seu aniversário?
Meu aniversário é em dezembro
Qual o seu nome amigão?
Sim, sou o Boop
nome
Boop.
nome
Boop.
dia
Nos vemos em breve!
noite
Nos vemos depois
vamos embora
Foi ótimo falar com você hoje
